In [10]:
import glob as gb
import cv2
import numpy as np
import random
from sklearn.externals import joblib
import requests

In [11]:
head_silhouette_path = gb.glob("head_test\\*.png")
whole_body_edge_path = gb.glob("fullbody_test\\*.png")
img_path=gb.glob("test\\*.jpg")
feature_list = []
gender_list=[]
ethnicity_list=[]
age_list=[]
length = []
whole_body_height_list = []

In [17]:
Url='https://api-cn.faceplusplus.com/facepp/v3/detect' 
URL='https://api-cn.faceplusplus.com/humanbodypp/v1/skeleton'
key = "1s-bG5PaBywr0DGIn7JMFDm3PWENN0il"    
secret = "a--zQAxt_M98ZxKrEzyGBPfJG-qZKord"
attributes="gender,age,ethnicity,eyegaze"
data={"api_key":key,"api_secret":secret,"return_attributes":attributes}
data2={"api_key":key,"api_secret":secret}

In [13]:
# Features of Head
feature_list = []
for path in head_silhouette_path:
    # for path in head_silhouette_path:
    img = cv2.imread(path, 0)
    height = img.shape[0]
    width= img.shape[1]
    feature_list.append(height)
    feature_list.append(width)

# Feature of whole body height
for path in whole_body_edge_path:
    img = cv2.imread(path, 0)
    height_of_whole_body=img.shape[0]
    whole_body_height_list.append(height_of_whole_body)

In [14]:
m=0
gender_list=[]
ethnicity_list=[]
age_list=[]
for path2 in img_path:
    m=m+1
    files={"image_file":open(path2,"rb")}
    response=requests.post(Url,data=data,files=files)
    reqcon=response.content.decode('utf-8')
    mydict = eval(reqcon)
    face=mydict['faces']
    age=face[0]['attributes']['age']['value']
    gender=face[0]['attributes']['gender']['value']
    ethnicty=face[0]['attributes']['ethnicity']['value']
    gender_list.append(gender)
    ethnicity_list.append(ethnicty)
    age_list.append(age)

In [15]:
for i in range(22):
    if (i%2)==0:
        gender_list[i]=gender_list[i+1]
        ethnicity_list[i]=ethnicity_list[i+1]
        age_list[i]=age_list[i+1]
        print(i)

0
2
4
6
8
10
12
14
16
18
20


In [18]:
# Compute Shoulder Width
m=0
shoulderWidth_list=[]
for path in img_path:
    m=m+1
    files={"image_file":open(path,"rb")}
    if m%2==0:
        response2=requests.post(URL,data=data2,files=files)
        reqcon2=response2.content.decode('utf-8')
        mydict2 = eval(reqcon2)
        rrx=mydict2['skeletons'][0]['landmark']['left_shoulder']['x']
        rry=mydict2['skeletons'][0]['landmark']['left_shoulder']['y']
        llx=mydict2['skeletons'][0]['landmark']['right_shoulder']['x']
        lly=mydict2['skeletons'][0]['landmark']['right_shoulder']['y']
        shoulderWidth=(rrx-llx)**2+(rry-lly)**2
        shoulderWidth_list.append(shoulderWidth)
        shoulderWidth_list.append(shoulderWidth)
        print(m-1,"and",m,"are ok")

1 and 2 are ok
3 and 4 are ok
5 and 6 are ok
7 and 8 are ok
9 and 10 are ok
11 and 12 are ok
13 and 14 are ok
15 and 16 are ok
17 and 18 are ok
19 and 20 are ok
21 and 22 are ok


In [19]:
gender_feature=np.array(gender_list)
gender_feature=gender_feature.reshape(22,1)

ethnicity_feature=np.array(ethnicity_list)
ethnicity_feature=ethnicity_feature.reshape(22,1)

age_feature=np.array(age_list)
age_feature=age_feature.reshape(22,1)

whole_body_height_matrix = np.array(whole_body_height_list)
whole_body_height_matrix = whole_body_height_matrix.reshape(22, 1)

feature_matrix = np.array(feature_list)
feature_matrix = feature_matrix.reshape(22, 2)

shoulderWidth_matrix = np.array(shoulderWidth_list)
shoulderWidth_matrix = shoulderWidth_matrix.reshape(22, 1)

feature = np.concatenate((feature_matrix, whole_body_height_matrix,gender_feature,ethnicity_feature,age_feature,shoulderWidth_matrix), axis=1)
print(feature)
print(feature.shape)

[['177' '177' '1218' 'Male' 'WHITE' '24' '53066']
 ['190' '147' '1222' 'Male' 'WHITE' '24' '53066']
 ['154' '173' '1131' 'Female' 'WHITE' '27' '32525']
 ['151' '155' '1133' 'Female' 'WHITE' '27' '32525']
 ['181' '202' '1206' 'Male' 'INDIA' '28' '54185']
 ['174' '143' '1205' 'Male' 'INDIA' '28' '54185']
 ['181' '157' '1215' 'Male' 'WHITE' '23' '47089']
 ['172' '148' '1223' 'Male' 'WHITE' '23' '47089']
 ['184' '177' '1272' 'Male' 'WHITE' '31' '57605']
 ['177' '154' '1280' 'Male' 'WHITE' '31' '57605']
 ['171' '172' '1297' 'Male' 'WHITE' '30' '50176']
 ['171' '146' '1305' 'Male' 'WHITE' '30' '50176']
 ['155' '159' '1169' 'Male' 'ASIAN' '25' '40085']
 ['163' '137' '1183' 'Male' 'ASIAN' '25' '40085']
 ['178' '164' '1224' 'Male' 'ASIAN' '22' '56169']
 ['171' '141' '1223' 'Male' 'ASIAN' '22' '56169']
 ['183' '170' '1222' 'Male' 'WHITE' '43' '58081']
 ['178' '150' '1215' 'Male' 'WHITE' '43' '58081']
 ['171' '164' '1243' 'Male' 'BLACK' '43' '60509']
 ['184' '147' '1244' 'Male' 'BLACK' '43' '6050

In [20]:
#encode some features
for i in range(22):
    if feature[i][3]=='Male':
        feature[i][3]=0
    if feature[i][3]=='Female':
        feature[i][3]=1
    if feature[i][4]=="ASIAN":
        feature[i][4]=1
    if feature[i][4]=="INDIA":
        feature[i][4]=2
    if feature[i][4]=="BLACK":
        feature[i][4]=3
    if feature[i][4]=="WHITE":
        feature[i][4]=4

In [21]:
print(feature)

[['177' '177' '1218' '0' '4' '24' '53066']
 ['190' '147' '1222' '0' '4' '24' '53066']
 ['154' '173' '1131' '1' '4' '27' '32525']
 ['151' '155' '1133' '1' '4' '27' '32525']
 ['181' '202' '1206' '0' '2' '28' '54185']
 ['174' '143' '1205' '0' '2' '28' '54185']
 ['181' '157' '1215' '0' '4' '23' '47089']
 ['172' '148' '1223' '0' '4' '23' '47089']
 ['184' '177' '1272' '0' '4' '31' '57605']
 ['177' '154' '1280' '0' '4' '31' '57605']
 ['171' '172' '1297' '0' '4' '30' '50176']
 ['171' '146' '1305' '0' '4' '30' '50176']
 ['155' '159' '1169' '0' '1' '25' '40085']
 ['163' '137' '1183' '0' '1' '25' '40085']
 ['178' '164' '1224' '0' '1' '22' '56169']
 ['171' '141' '1223' '0' '1' '22' '56169']
 ['183' '170' '1222' '0' '4' '43' '58081']
 ['178' '150' '1215' '0' '4' '43' '58081']
 ['171' '164' '1243' '0' '3' '43' '60509']
 ['184' '147' '1244' '0' '3' '43' '60509']
 ['155' '145' '1345' '0' '1' '41' '44645']
 ['212' '182' '1345' '0' '1' '41' '44645']]


In [41]:
joblib.dump(feature, 'feature_test.pkl')

['feature_test.pkl']